In [1]:
#pip install git+https://github.com/deanmalmgren/textract.git

In [2]:
import textract

In [3]:
import pandas as pd
import numpy as np
import re

In [4]:
import tabula

In [34]:
#extraigo texto
archivo= 'tresalia\Bancos\Banamex_nacional\pdf\cuenta_perfiles_banamex.pdf'

In [35]:
text = textract.process(archivo)

In [36]:
#cambio formato
encoding = 'utf-8'
text_str=text.decode(encoding)

In [37]:
#separo
pdf=re.split(r"[~\r\n]+", text_str)

In [38]:
information_pdf=pd.DataFrame(pdf).rename(columns={0:"informacion"})
information_pdf.head()

,informacion
0,cffibanamex@
1,El Banco Nacional de México
2,Página 1 de 11
3,g NOMBRE APELLIDO P APELLIDO M
4,"..,. DIRECCIÓN ""' COLONIA"


In [39]:
#extraigo nombre de cliente
cliente_nombre_list=[]
for index,row in information_pdf.iterrows():
    if row["informacion"].startswith("g"):
        cliente_nombre=row[0]
        cliente_nombre_list.append(cliente_nombre)
    

In [40]:
cliente_nombre_list

['g NOMBRE APELLIDO P  APELLIDO M']

In [106]:
cliente_nombre=(cliente_nombre_list[0].replace('g','')).strip()

In [107]:
cliente_nombre

'NOMBRE APELLIDO P  APELLIDO M'

In [43]:
#obtengo periodo

In [44]:
periodo_list=[]
for index,row in information_pdf.iterrows():
    if row["informacion"].startswith("Período") or row["informacion"].startswith("período") or row["informacion"].startswith("Periodo") or row["informacion"].startswith("periodo"):
        periodo=row[0]
        periodo_list.append(periodo)

In [45]:
periodo

'Período del 5 de septiembre al 4 de octubre del 2020'

In [46]:
periodo=[]
for i in pdf:
    if i.startswith("Período") or i.startswith("período") or i.startswith("Periodo") or i.startswith("periodo"):
        periodo.append(i)
        
print(periodo)

['Período del 5 de septiembre al 4 de octubre del 2020']


In [47]:
informacion=[]
for i in range(len(pdf)):
    if re.search('contrato',pdf[i]):
        informacion.append(pdf[i])
        informacion.append(pdf[i+1])

print(informacion)

['Número de contrato Número de sucursal Número de cuenta de cheques Número de Tarjeta de Débito CLABE lnterbancaria Número de cliente RFC', '9999999999 9999 99999999999 9999999999999999 999999999999999999 99999999 XXXX999999X99', 'Número de contrato', '9999999999', 'Tambien puedes realizar dep6sitos en cajeros automaticos y sucursales Cltibanamex. En caso de no estar de acuerdo con las modificaciones senaladas, podras solicitar la terminaci6n del Contrato dentro de los 30 dfas posteriore a Que surta electo la presente notificaci6n, por lo que durante ese pedodo nose aplicara el cobra de las comisiones mencionadas previamente. Se entendera que aceptas las modificaciones efectuadas a las comisiones, si no solicitas la terminaci6n del contrato dentro del plazo seiialado. Tu derecho a dar por terminado el Contrato en cualquier otro momenta seguira siendo vigente.', 'Atentamente.', 'Tu contrato de Banca lectr6nica tiene na nueva actualizaci6n', 'Te inlormarnos que tu contrato de prestacion 

In [48]:
nombre=informacion[0].split('Número')
nombre.pop(0)
nombre[3]='Tarjeta de Débito'
nombre[4]='CLABE'
nombre.append('# cliente')
nombre.append('RFC')


In [49]:
nombre

[' de contrato ',
 ' de sucursal ',
 ' de cuenta de cheques ',
 'Tarjeta de Débito',
 'CLABE',
 '# cliente',
 'RFC']

In [50]:
valores=informacion[1].split(' ')

In [51]:
val_dict= dict(zip(nombre,valores))

In [52]:
val_dict

{' de contrato ': '9999999999',
 ' de sucursal ': '9999',
 ' de cuenta de cheques ': '99999999999',
 'Tarjeta de Débito': '9999999999999999',
 'CLABE': '999999999999999999',
 '# cliente': '99999999',
 'RFC': 'XXXX999999X99'}

In [53]:
pdf

['cffibanamex@',
 'El Banco Nacional de México',
 'Página 1 de 11',
 'g NOMBRE APELLIDO P  APELLIDO M',
 '..,. DIRECCIÓN "\' COLONIA',
 'IN',
 '00000 DELEGACIÓN, CIUDAD DE MEXICO C.R.11801',
 'Estado de Cuenta',
 'Cuenta Perfiles',
 'Período del 5 de septiembre al 4 de octubre del 2020',
 'Fecha de Corte',
 '02 de octubre de 2020',
 'Número de contrato Número de sucursal Número de cuenta de cheques Número de Tarjeta de Débito CLABE lnterbancaria Número de cliente RFC',
 '9999999999 9999 99999999999 9999999999999999 999999999999999999 99999999 XXXX999999X99',
 'GAT Nominal y Real antes de impuestos Interés aplicable o Rendimientos Comisiones efectivamente cobradas',
 'Comportamiento',
 'No Aplica $0.00',
 '$105.00',
 'Resumen En Pesos Moneda Nacional',
 'Saldo anterior Depósitos Retiros en efectivo Otros cargos Saldo al corte Saldo promedio mensuaf',
 '1,291.41 0.00 0.00',
 '5,538.45 5,647.96 2,065.77',
 'Saldo anterior',
 'o',
 'Depósitos',
 'Retiros en efectivo',
 '$0',
 'Comisiones $

In [54]:
#movimientos
movimientos=[]
for i in range(len(pdf)):
    if pdf[i].startswith('Detalle'):
        movimientos.append(i)

In [55]:
movimientos

[94, 124, 160]

In [56]:
#fin de movimientos
fin=[]
for i in range(len(pdf)):
    if pdf[i].startswith('Resumen Operaciones Tarjeta'):
        fin.append(i)

In [57]:
fin

[190]

In [58]:
tabla_todo=pdf[movimientos[0]:fin[0]]

In [59]:
tabla_pag_1=pdf[movimientos[0]:movimientos[1]]

In [60]:
tabu=tabula.read_pdf(archivo, pages='all')

# PAGINA 1

In [61]:
#fechas
fecha_todo=[]
for i in range(len(tabla_todo)):
    if re.match('fecha',tabla_todo[i].lower()):
        fecha_todo.append(tabla_todo[i])
    elif re.match(r'\d{2} \w{3}',tabla_todo[i]):
        fecha_todo.append(tabla_todo[i])

        
fecha_todo

['FECHA 05 SEP 07 SEP 08 SEP',
 '10 SEP 10 SEP 10 SEP 18 SEP 18 SEP',
 '18 SEP',
 'FECHA 21 SEP 21 SEP 21 SEP 22 SEP',
 '24 SEP 24 SEP 24 SEP 25 SEP 25 SEP',
 '25 SEP',
 '28 SEP',
 'FECHA 30 SEP 30 SEP',
 '02 OCT 02 OCT']

In [62]:
#separo
fechas_todo_2=[]

for i in fecha_todo:
    fechas_todo_2.append(re.findall(r'\d{2} \w{3}',i))

In [63]:
#aplano lista de listas
fechas_todo_3 = []
for i in fechas_todo_2:
    for j in i:
        fechas_todo_3.append(j)

In [64]:
len(fechas_todo_3)

24

In [65]:
fechas_df=fechas_todo_3

In [66]:
#conceptos
tabla_string1=" ".join(tabla_pag_1)

In [67]:
tabla_string1

'Detalle de Operaciones En pesos Moneda Nacional FECHA 05 SEP 07 SEP 08 SEP 10 SEP 10 SEP 10 SEP 18 SEP 18 SEP 18 SEP CONCEPTO SALDO ANTERIOR COBRO DE CHEQUE NUMERO 000 SUC 0000 CAJA 0000 AUT 00000000 HORA 15:41 EXENCION COBRO COMISION MANEJO DE CUENTA PERIODO AGO05 AL SEP04 IMPORTE A EX ENTAR $165.00 CAJA 0000 AUT 00000000 HORA 03:09 SUC 0000 COBRO DE CHEQUE NUMERO 000 suc 0000 CAJA 0000 AUT 00000000 HORA 14:30 COBRO DE CHEQUE NUMERO 174 suc 0000 CAJA 0000 AUT 00000000 HORA 14:49 COBRO DE CHEQUE NUMERO 000 suc 0000 CAJA 0000 AUT 00000000 HORA 14:50 GNC PLAZA LILAS MAX0611157H8 9999 999999999999999999999999 99999999 CAJA 0093 AUT 99999999 99999999 HORA 06:48 SUC 9999 XXXX XXXX & XXX XX 000000000 0000 suc 0342 CAJA 0000 AUT 00000000 HORA 06:48 00000000 00000000000000000000000 00000000 suc 0000 CAJA 0000 AUT 00000000 HORA 06:48 LILAS 000000000 0006475 Envía y recibe dinero al instante sin dar de alta una cuenta, hazlo desde CoDi(!) en Citibanamex Móvil Maneja tu dinero desde la App. RETI

In [68]:
conceptos_p1=re.split('SALDO |EXCENSINCION |COB|XXXX XXXX|GN',tabla_string1)
conceptos_p1

['Detalle de Operaciones En pesos Moneda Nacional FECHA 05 SEP 07 SEP 08 SEP 10 SEP 10 SEP 10 SEP 18 SEP 18 SEP 18 SEP CONCEPTO ',
 'ANTERIOR ',
 'RO DE CHEQUE NUMERO 000 SUC 0000 CAJA 0000 AUT 00000000 HORA 15:41 EXENCION ',
 'RO COMISION MANEJO DE CUENTA PERIODO AGO05 AL SEP04 IMPORTE A EX ENTAR $165.00 CAJA 0000 AUT 00000000 HORA 03:09 SUC 0000 ',
 'RO DE CHEQUE NUMERO 000 suc 0000 CAJA 0000 AUT 00000000 HORA 14:30 ',
 'RO DE CHEQUE NUMERO 174 suc 0000 CAJA 0000 AUT 00000000 HORA 14:49 ',
 'RO DE CHEQUE NUMERO 000 suc 0000 CAJA 0000 AUT 00000000 HORA 14:50 ',
 'C PLAZA LILAS MAX0611157H8 9999 999999999999999999999999 99999999 CAJA 0093 AUT 99999999 99999999 HORA 06:48 SUC 9999 ',
 ' & XXX XX 000000000 0000 suc 0342 CAJA 0000 AUT 00000000 HORA 06:48 00000000 00000000000000000000000 00000000 suc 0000 CAJA 0000 AUT 00000000 HORA 06:48 LILAS 000000000 0006475 Envía y recibe dinero al instante sin dar de alta una cuenta, hazlo desde CoDi(!) en Citibanamex Móvil Maneja tu dinero desde la 

In [69]:
conceptos_pag_1=[]
for i in range(len(conceptos_p1)):
    if conceptos_p1[i].startswith('ANTERIOR'):
        j='SALDO '+conceptos_p1[i]
        conceptos_pag_1.append(j)
    elif conceptos_p1[i].startswith('RO COMISION'):
        j='EXCENSION COB'+conceptos_p1[i]
        conceptos_pag_1.append(j)
    elif conceptos_p1[i].startswith('RO'):
        j='COB'+conceptos_p1[i]
        conceptos_pag_1.append(j)
    elif conceptos_p1[i].startswith('C'):
        j='GN'+conceptos_p1[i]
        conceptos_pag_1.append(j)
    elif conceptos_p1[i].startswith(' &'):
        j='XXXX XXXX'+conceptos_p1[i]
        conceptos_pag_1.append(j)
conceptos_pag_1

['SALDO ANTERIOR ',
 'COBRO DE CHEQUE NUMERO 000 SUC 0000 CAJA 0000 AUT 00000000 HORA 15:41 EXENCION ',
 'EXCENSION COBRO COMISION MANEJO DE CUENTA PERIODO AGO05 AL SEP04 IMPORTE A EX ENTAR $165.00 CAJA 0000 AUT 00000000 HORA 03:09 SUC 0000 ',
 'COBRO DE CHEQUE NUMERO 000 suc 0000 CAJA 0000 AUT 00000000 HORA 14:30 ',
 'COBRO DE CHEQUE NUMERO 174 suc 0000 CAJA 0000 AUT 00000000 HORA 14:49 ',
 'COBRO DE CHEQUE NUMERO 000 suc 0000 CAJA 0000 AUT 00000000 HORA 14:50 ',
 'GNC PLAZA LILAS MAX0611157H8 9999 999999999999999999999999 99999999 CAJA 0093 AUT 99999999 99999999 HORA 06:48 SUC 9999 ',
 'XXXX XXXX & XXX XX 000000000 0000 suc 0342 CAJA 0000 AUT 00000000 HORA 06:48 00000000 00000000000000000000000 00000000 suc 0000 CAJA 0000 AUT 00000000 HORA 06:48 LILAS 000000000 0006475 Envía y recibe dinero al instante sin dar de alta una cuenta, hazlo desde CoDi(!) en Citibanamex Móvil Maneja tu dinero desde la App. RETIROS DEPÓSITOS 2,000.00 950.00 1 686.00 1,326.00 2,029.70 350.00 000190.B25INDA0I

In [70]:
len(conceptos_pag_1)

8

In [71]:
#retiros
retiros_lista_pag_1=[]
for i in range(len(tabla_pag_1)):
    if re.match('retiros',tabla_pag_1[i].lower()):
        retiros_lista_pag_1.append(tabla_pag_1[i])
        retiros_lista_pag_1.append(tabla_pag_1[i+1])
        retiros_lista_pag_1.append(tabla_pag_1[i+2])
        retiros_lista_pag_1.append(tabla_pag_1[i+3])
        retiros_lista_pag_1.append(tabla_pag_1[i+4])
retiros_lista_pag_1

['RETIROS',
 'DEPÓSITOS',
 '2,000.00',
 '950.00 1 686.00 1,326.00 2,029.70',
 '350.00']

In [72]:
retiros1_1=[]
for i in retiros_lista_pag_1:
    if re.search(r'\d+', i):
        retiros1_1.append(i)

In [73]:
retiros1_1

['2,000.00', '950.00 1 686.00 1,326.00 2,029.70', '350.00']

In [74]:
retiros1_2=[]
for i in retiros1_1:
    retiros1_2.append(i.split('.00'))
    
retiros1_2

[['2,000', ''], ['950', ' 1 686', ' 1,326', ' 2,029.70'], ['350', '']]

In [75]:
retiros1_3=[]
for i in retiros1_2:
    for j in i:
        if re.search(r'\d+', j):
            retiros1_3.append(j+'.00')
        
retiros1_3

['2,000.00', '950.00', ' 1 686.00', ' 1,326.00', ' 2,029.70.00', '350.00']

In [76]:
#organizo retiros pagina 1
retiros_pag_1=[]
contador=0
for i in range(len(conceptos_pag_1)):
    if conceptos_pag_1[i].startswith('COBRO'):
        retiros_pag_1.append(retiros1_3[contador])
        contador=contador+1
    
    elif conceptos_pag_1[i].startswith('GNC'):
        retiros_pag_1.append(retiros1_3[contador])
        contador=contador+1
        
    elif conceptos_pag_1[i].startswith('XXX'):
        retiros_pag_1.append(retiros1_3[contador])
        contador=contador+1
    
    else:
        retiros_pag_1.append(0)
retiros_pag_1

[0,
 '2,000.00',
 0,
 '950.00',
 ' 1 686.00',
 ' 1,326.00',
 ' 2,029.70.00',
 '350.00']

In [77]:
#depositos
depositos_lista_pag_1=[]
for i in range(len(tabla_pag_1)):
    if re.match('depósitos',tabla_pag_1[i].lower()):
        depositos_lista_pag_1.append(tabla_pag_1[i])
        
depositos_lista_pag_1
#no hay depositos

['DEPÓSITOS']

In [78]:
depositos1=[]
for i in depositos_lista_pag_1:
    if re.search(r'\d+', i):
        depositos1.append(i)

depositos1_1=[]     
for i in depositos1:
    j=i.split(' ')
    for k in j:
        depositos1_1.append(k)

In [79]:
#organizo depositos
depositos_pag_1=[]
contador=0
for i in range(len(conceptos_pag_1)):
    if conceptos_pag_1[i].startswith('DEPOSITO') or conceptos_pag_1[i].startswith('INTERES'):
        depositos_pag_1.append(depositos1_1[contador])
        contador=contador+1
    else:
        depositos_pag_1.append('0')
depositos_pag_1

['0', '0', '0', '0', '0', '0', '0', '0']

In [80]:
#saldos
saldos_lista_pag_1=[]
for i in range(len(tabla_pag_1)):
    if re.match('saldo',tabla_pag_1[i].lower()):
        saldos_lista_pag_1.append(tabla_pag_1[i])
        saldos_lista_pag_1.append(tabla_pag_1[i+1])
        saldos_lista_pag_1.append(tabla_pag_1[i+2])
        
saldos_lista_pag_1

['SALDO ANTERIOR COBRO DE CHEQUE NUMERO 000 SUC 0000 CAJA 0000 AUT 00000000 HORA 15:41 EXENCION COBRO COMISION MANEJO DE CUENTA PERIODO AGO05 AL SEP04 IMPORTE A EX ENTAR $165.00 CAJA 0000 AUT 00000000 HORA 03:09 SUC 0000 COBRO DE CHEQUE NUMERO 000',
 'suc 0000',
 'CAJA 0000 AUT 00000000 HORA 14:30 COBRO DE CHEQUE NUMERO 174',
 'SALDO 1 291.41 9,291.41 9,291.41 8,341.41 6 655.41 5,329.41 3,299.71 2 949.71',
 '\x0ccffibanamex@',
 'NOMBRE APELLIDO P APELLIDO M']

In [81]:
saldos1_1=[]
for i in range(len(saldos_lista_pag_1)):
    if re.search(r'SALDO \d+', saldos_lista_pag_1[i]):
        saldos1_1.append(saldos_lista_pag_1[i])
        saldos1_1.append(saldos_lista_pag_1[i+1])
        
saldos1_1

['SALDO 1 291.41 9,291.41 9,291.41 8,341.41 6 655.41 5,329.41 3,299.71 2 949.71',
 '\x0ccffibanamex@']

In [82]:
saldos1_2=[]
for i in saldos1_1:
    j=re.split('SALDO|\.\d{2}',i)
    saldos1_2.append(j)
    
saldos1_3=[]
for i in saldos1_2:
    for j in i:
        if re.search(r'\d+',j):
            saldos1_3.append(j)
            
saldos1_3

[' 1 291',
 ' 9,291',
 ' 9,291',
 ' 8,341',
 ' 6 655',
 ' 5,329',
 ' 3,299',
 ' 2 949']

In [83]:
saldos_pag_1= saldos1_3

# con tabula

In [84]:
tabu[1]

,Detalle de Operaciones En pesos Moneda Nacional,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,FECHA CONCEPTO,NaN,RETIROS,DEPÓSITOS,SALDO
1,5411 75412910263975987909057,NaN,NaN,NaN,NaN
2,20200921 CAJA 0093 AUT 02985979,NaN,NaN,NaN,NaN
3,20200921 HORA 06:48 SUC 0342,NaN,234.00,NaN,"2,715.71"
4,21 SEP REPOSTERIA,NaN,NaN,NaN,NaN
5,XXXX999999XX9 5441,NaN,NaN,NaN,NaN
6,20200923 75326900265000784368680,NaN,NaN,NaN,NaN
7,20200923 suc 0342,NaN,NaN,NaN,NaN
8,CAJA 0093 AUT 01011285 HORA 05:30 CITY,NaN,1 360.00,NaN,1 355.71
9,21 SEP MARKET,NaN,NaN,NaN,NaN


In [85]:
conceptos_lista=[]
for i in range(len(tabu)):
    try:
        for j in tabu[i]['Detalle de Operaciones En pesos Moneda Nacional']:
            conceptos_lista.append(str(j))
    except:
        pass
    

In [86]:
conceptos_str=' '.join(conceptos_lista)

In [87]:
mes=re.findall(r'\w{3}',fechas_todo_3[0])

In [88]:
conceptos_2_y_3=re.split(r'\d{2} ENE |\d{2} FEB |\d{2} MAR |\d{2} ABR|\d{2} MAY |\d{2} JUN |\d{2} JUL |\d{2} SEP |\d{2} OCT |\d{2} NOV |\d{2} DEC '
                     ,conceptos_str)

In [89]:
conceptos_2_y_3

['FECHA CONCEPTO 5411 75412910263975987909057 20200921 CAJA 0093 AUT 02985979 20200921 HORA 06:48 SUC 0342 ',
 'REPOSTERIA XXXX999999XX9 5441 20200923 75326900265000784368680 20200923 suc 0342 CAJA 0093 AUT 01011285 HORA 05:30 CITY ',
 'MARKET 5411 75435290266536760465142 20200923 CAJA 0093 AUT 01011287 20200923 HORA 05:30 SUC 0342 ',
 'FLORES  5992 20200923 75412780265000108700221 20200923 suc 0342 CAJA 0093 AUT 01011289 HORA 05:30 ',
 'TERMINAL 131210UU7 5811 suc 0342 CAJA 0093 AUT 00881422 HORA 05:30 20200924 75445500266001282364505 20200924 suc 0342 nan ',
 'CAJA 0093 AUT 00881422 HORA 05:30 CITY MARKET 5411 75435290269536760467046 20200928 CAJA 0093 AUT 00906083 nan 20200928 HORA 05:40 SUC 0342 ',
 'COBRO DE CHEQUE NUMERO 999 SUC 4409 CAJA 0003 AUT 00451934 HORA 12:19 ',
 'COBRO DE CHEQUE NUMERO 999 SUC 4409 CAJA 0003 AUT 00451948 HORA 12:32 ',
 'COBRO DE CHEQUE NUMERO 999 SUC 4409 CAJA 0001 AUT 00017534 HORA 11 :29 ',
 'UBER *EATS HELP.UBER.COM 5812 20200928 152763202700024874939

In [90]:
conceptos_df=conceptos_pag_1+conceptos_2_y_3

In [91]:
#retiros
retiros_lista=[]
for i in range(len(tabu)):
    tabu[i].fillna(0,inplace=True)
    try:
        for j in tabu[i]['Unnamed: 1']:
            if j!=0:
                if j!='RETIROS':
                    retiros_lista.append(str(j))
    except:
        pass
    

In [92]:
retiros_df= retiros_pag_1+retiros_lista

In [93]:
#saldoss
saldos_lista=[]
for i in range(len(tabu)):
    tabu[i].fillna(0,inplace=True)
    try:
        for j in tabu[i]['Unnamed: 3']:
            if j!=0:
                if j!='SALDO':
                    saldos_lista.append(str(j))
    except:
        pass
    

In [94]:
len(saldos_lista)

16

In [95]:
saldos_df=saldos1_3+saldos_lista

In [96]:
depositos_df=[]
for i in retiros_df:
    if i!=0:
        depositos_df.append(0)
    elif i==0:
        depositos_df.append(0)
    else:
        depositos_df.append('pendiente')

In [97]:
len(depositos_df)

24

In [98]:
estado=pd.DataFrame(list(zip(fechas_todo_3, conceptos_df, retiros_df, depositos_df, saldos_df)),
              columns=['fecha','concepto','retiro', 'deposito','saldo'])

In [99]:
estado

,fecha,concepto,retiro,deposito,saldo
0,05 SEP,SALDO ANTERIOR,0,0,1 291
1,07 SEP,COBRO DE CHEQUE NUMERO 000 SUC 0000 CAJA 0000 ...,"2,000.00",0,"9,291"
2,08 SEP,EXCENSION COBRO COMISION MANEJO DE CUENTA PERI...,0,0,"9,291"
3,10 SEP,COBRO DE CHEQUE NUMERO 000 suc 0000 CAJA 0000 ...,950.00,0,"8,341"
4,10 SEP,COBRO DE CHEQUE NUMERO 174 suc 0000 CAJA 0000 ...,1 686.00,0,6 655
5,10 SEP,COBRO DE CHEQUE NUMERO 000 suc 0000 CAJA 0000 ...,"1,326.00",0,"5,329"
6,18 SEP,GNC PLAZA LILAS MAX0611157H8 9999 999999999999...,"2,029.70.00",0,"3,299"
7,18 SEP,XXXX XXXX & XXX XX 000000000 0000 suc 0342 CAJ...,350.00,0,2 949
8,18 SEP,FECHA CONCEPTO 5411 75412910263975987909057 20...,234.00,0,"2,715.71"
9,21 SEP,REPOSTERIA XXXX999999XX9 5441 20200923 7532690...,1 360.00,0,1 355.71


In [100]:
estado['banco']='banamex'

In [101]:
estado['nacionalidad']='nacional'

In [103]:
periodo

['Período del 5 de septiembre al 4 de octubre del 2020']

In [104]:
estado['fecha de corte']= periodo[0]

In [109]:
estado['titular']=cliente_nombre

In [111]:
val_dict

{' de contrato ': '9999999999',
 ' de sucursal ': '9999',
 ' de cuenta de cheques ': '99999999999',
 'Tarjeta de Débito': '9999999999999999',
 'CLABE': '999999999999999999',
 '# cliente': '99999999',
 'RFC': 'XXXX999999X99'}

In [113]:
estado['cuenta']=val_dict[' de cuenta de cheques ']

In [114]:
estado['saldo inicial']=estado['saldo'][0]

In [115]:
if re.search('usd',text_str.lower()) or re.search('dolar', text_str.lower()) or re.search('dollar', text_str.lower()):
    moneda='usd'
else:
    moneda='mxn'

In [116]:
moneda

'mxn'

In [117]:
estado['divisa']=moneda

In [118]:
edo_cuenta= estado[['banco','nacionalidad','fecha de corte', 'titular','cuenta','fecha','concepto','saldo inicial',
                    'retiro','deposito','saldo','divisa']]

In [128]:
nombre_archivo=re.findall(r'\d+ \w+ \w+ \w+ \d+ \w+ \w+ \w+ \d+',periodo[0])[0]+'-banamex-cuenta-perfiles'
nombre_archivo=nombre_archivo.replace(' ','-')
nombre_archivo

'5-de-septiembre-al-4-de-octubre-del-2020-banamex-cuenta-perfiles'

In [129]:
#exporto
#edo_cuenta.to_csv('tresalia\Bancos\Banamex_nacional\salida\\'+nombre_archivo+'.txt' , sep='\t', index=False)

In [130]:
edo_cuenta

,banco,nacionalidad,fecha de corte,titular,cuenta,fecha,concepto,saldo inicial,retiro,deposito,saldo,divisa
0,banamex,nacional,Período del 5 de septiembre al 4 de octubre de...,NOMBRE APELLIDO P APELLIDO M,99999999999,05 SEP,SALDO ANTERIOR,1 291,0,0,1 291,mxn
1,banamex,nacional,Período del 5 de septiembre al 4 de octubre de...,NOMBRE APELLIDO P APELLIDO M,99999999999,07 SEP,COBRO DE CHEQUE NUMERO 000 SUC 0000 CAJA 0000 ...,1 291,"2,000.00",0,"9,291",mxn
2,banamex,nacional,Período del 5 de septiembre al 4 de octubre de...,NOMBRE APELLIDO P APELLIDO M,99999999999,08 SEP,EXCENSION COBRO COMISION MANEJO DE CUENTA PERI...,1 291,0,0,"9,291",mxn
3,banamex,nacional,Período del 5 de septiembre al 4 de octubre de...,NOMBRE APELLIDO P APELLIDO M,99999999999,10 SEP,COBRO DE CHEQUE NUMERO 000 suc 0000 CAJA 0000 ...,1 291,950.00,0,"8,341",mxn
4,banamex,nacional,Período del 5 de septiembre al 4 de octubre de...,NOMBRE APELLIDO P APELLIDO M,99999999999,10 SEP,COBRO DE CHEQUE NUMERO 174 suc 0000 CAJA 0000 ...,1 291,1 686.00,0,6 655,mxn
5,banamex,nacional,Período del 5 de septiembre al 4 de octubre de...,NOMBRE APELLIDO P APELLIDO M,99999999999,10 SEP,COBRO DE CHEQUE NUMERO 000 suc 0000 CAJA 0000 ...,1 291,"1,326.00",0,"5,329",mxn
6,banamex,nacional,Período del 5 de septiembre al 4 de octubre de...,NOMBRE APELLIDO P APELLIDO M,99999999999,18 SEP,GNC PLAZA LILAS MAX0611157H8 9999 999999999999...,1 291,"2,029.70.00",0,"3,299",mxn
7,banamex,nacional,Período del 5 de septiembre al 4 de octubre de...,NOMBRE APELLIDO P APELLIDO M,99999999999,18 SEP,XXXX XXXX & XXX XX 000000000 0000 suc 0342 CAJ...,1 291,350.00,0,2 949,mxn
8,banamex,nacional,Período del 5 de septiembre al 4 de octubre de...,NOMBRE APELLIDO P APELLIDO M,99999999999,18 SEP,FECHA CONCEPTO 5411 75412910263975987909057 20...,1 291,234.00,0,"2,715.71",mxn
9,banamex,nacional,Período del 5 de septiembre al 4 de octubre de...,NOMBRE APELLIDO P APELLIDO M,99999999999,21 SEP,REPOSTERIA XXXX999999XX9 5441 20200923 7532690...,1 291,1 360.00,0,1 355.71,mxn
